In [150]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [151]:
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

df_train = pd.DataFrame(titanic)

pd.set_option('display.max_rows', 4)  # Show 4 rows
pd.set_option('display.max_columns', None)  # Show all columns

# Dataset 1

Dropping     Cabin, Ticket, Name.

In [152]:
pd.set_option('display.max_rows', 4)  # Show all rows

df_train1 = df_train

df_train1.drop(columns = ['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

df_train1


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,male,26.0,0,0,30.0000,C
890,891,0,3,male,32.0,0,0,7.7500,Q


In [153]:
df_train1['Embarked'].fillna('S', inplace=True) # Two NaN in Embarked column... index 61 829 both survived

In [154]:
df_train1['Sex'] = df_train1['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2

In [155]:
df_train1['Embarked'] = df_train1['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

df_train1   

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1
1,2,1,1,2,38.0,1,0,71.2833,2
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2
890,891,0,3,1,32.0,0,0,7.7500,3


In [156]:
#Handling missing age

df_train1_age_Na = df_train1[df_train1['Age'].isna()]

df_train1 = df_train1.dropna(subset=['Age'])

In [157]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
...,...,...,...,...,...,...,...,...,...
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [158]:
df_train1_age = df_train1

df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']

/tmp/ipykernel_33/2467646141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']


In [159]:
df_train1

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,1,0,3,1,22.0,1,0,7.2500,1,1
1,2,1,1,2,38.0,1,0,71.2833,2,1
...,...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2,0
890,891,0,3,1,32.0,0,0,7.7500,3,0


In [160]:
#Model to predict age.

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

In [556]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId', 'Survived'])
y_age = df_train1_age['Age']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler_X.fit_transform(X_age)
y_age_scaled = scaler_y.fit_transform(y_age.values.reshape(-1, 1)).ravel()


X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

In [557]:
model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=28)

In [15]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor instance
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [163]:
y_pred = best_rf.predict(X_test)

y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 9.147076376427146
Mean Squared Error: 141.1009020333041
R-squared: 0.3568452399753249


In [105]:
y_pred_round = np.round(y_pred)

In [106]:
# now for the missing data.
pd.set_option('display.max_rows', 6) 
df_train1_age_Na


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
19,20,1,3,2,NaN,0,0,7.2250,2
...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,NaN,0,0,9.5000,1
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [107]:
df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']

/tmp/ipykernel_33/2457252628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']


In [108]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
5,6,0,3,1,NaN,0,0,8.4583,3,0
17,18,1,2,1,NaN,0,0,13.0000,1,0
19,20,1,3,2,NaN,0,0,7.2250,2,0
...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,NaN,0,0,9.5000,1,0
878,879,0,3,1,NaN,0,0,7.8958,1,0
888,889,0,3,2,NaN,1,2,23.4500,1,3


In [109]:
X_age = df_train1_age_Na.drop(columns = ['Age', 'PassengerId', 'Survived'])


X_age_scaled = scaler.fit_transform(X_age)

y_pred_na = best_rf.predict(X_age_scaled)


In [110]:
y_pred_na = y_pred_na.reshape(-1, 1)
y_pred_na = scaler_y.inverse_transform(y_pred_na)

In [111]:
y_pred_na

array([[22.28019565],
       [35.43711955],
       [25.58439923],
       [24.21751474],
       [19.73647338],
       [23.52711513],
       [31.67732475],
       [20.15863771],
       [24.21751474],
       [23.52711513],
       [22.89014313],
       [25.44991358],
       [20.15863771],
       [29.5564045 ],
       [38.18265376],
       [42.72733792],
       [22.01529416],
       [23.52711513],
       [22.89014313],
       [20.15863771],
       [22.89014313],
       [22.89014313],
       [23.52711513],
       [23.78237694],
       [22.93179952],
       [22.89014313],
       [22.66590941],
       [23.47778358],
       [28.68044994],
       [23.98633687],
       [23.32206122],
       [15.87749074],
       [26.06346668],
       [42.78943478],
       [14.21416109],
       [16.04557593],
       [34.41608855],
       [38.68481007],
       [22.72309122],
       [22.66590941],
       [20.15863771],
       [15.87749074],
       [21.23325287],
       [23.52711513],
       [13.73098548],
       [25

In [112]:
y_pred_na = np.round(y_pred_na)

In [113]:
df_train1_age_Na.loc[:, 'Age'] = y_pred_na


/tmp/ipykernel_33/2722028552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na.loc[:, 'Age'] = y_pred_na


In [114]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
5,6,0,3,1,22.0,0,0,8.4583,3,0
17,18,1,2,1,35.0,0,0,13.0000,1,0
19,20,1,3,2,26.0,0,0,7.2250,2,0
...,...,...,...,...,...,...,...,...,...,...
868,869,0,3,1,28.0,0,0,9.5000,1,0
878,879,0,3,1,24.0,0,0,7.8958,1,0
888,889,0,3,2,29.0,1,2,23.4500,1,3


In [115]:
pd.set_option('display.max_rows', 10)

df_train1_age_Na['original_index'] = df_train1_age_Na.index
df_train1['original_index'] = df_train1.index

/tmp/ipykernel_33/2977192852.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na['original_index'] = df_train1_age_Na.index
/tmp/ipykernel_33/2977192852.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['original_index'] = df_train1.index


In [116]:
merged_df = pd.concat([df_train1_age_Na, df_train1], axis=0)

In [117]:
merged_df.sort_values(by='original_index', inplace=True)

In [118]:
pd.set_option('display.max_rows', 10)

merged_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,original_index
0,1,0,3,1,22.0,1,0,7.2500,1,1,0
1,2,1,1,2,38.0,1,0,71.2833,2,1,1
2,3,1,3,2,26.0,0,0,7.9250,1,0,2
3,4,1,1,2,35.0,1,0,53.1000,1,1,3
4,5,0,3,1,35.0,0,0,8.0500,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,1,0,886
887,888,1,1,2,19.0,0,0,30.0000,1,0,887
888,889,0,3,2,29.0,1,2,23.4500,1,3,888
889,890,1,1,1,26.0,0,0,30.0000,2,0,889


In [119]:
df_version1 = merged_df.drop(columns = ['PassengerId', 'original_index'])

In [207]:
df_version1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,0,3,1,22.0,1,0,7.2500,1,1
1,1,1,2,38.0,1,0,71.2833,2,1
...,...,...,...,...,...,...,...,...,...
889,1,1,1,26.0,0,0,30.0000,2,0
890,0,3,1,32.0,0,0,7.7500,3,0


In [201]:
#Grid Search:
#Grid search exhaustively searches through a specified set of hyperparameter combinations. Here's how you can use grid search:

#Randomized Search:
#Randomized search explores a random subset of the hyperparameter space. This can be more efficient when the hyperparameter space is large.

In [572]:
X_v1 = df_version1.drop(columns = ['Survived'])
y_v1 = df_version1['Survived']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
#scaler = MinMaxScaler()


X_v1 = scaler_X.fit_transform(X_v1)
y_v1 = scaler_y.fit_transform(y_v1.values.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size = 0.2,random_state = 28)

In [573]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X_v1 = df_version1.drop(columns = ['Survived'])
y_v1 = df_version1['Survived']

scaler_X = StandardScaler()
X_v1 = scaler_X.fit_transform(X_v1)
X_train, X_test, y_train, y_test = train_test_split(X_v1, y_v1, test_size = 0.2,random_state = 28)


model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [440]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


In [217]:
y_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0])

In [125]:
y_test

287    0
621    1
395    0
273    0
155    0
      ..
405    0
220    1
585    1
817    0
55     1
Name: Survived, Length: 179, dtype: int64

# Prediction

In [534]:
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

df_test = pd.DataFrame(titanic_test)

In [535]:
pd.set_option('display.max_rows', 6)  # Show 4 rows

df_test_pure = df_test

df_test_pure

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [536]:
# Assuming df is your DataFrame
nan_counts = df_test_pure.isna().sum()


pd.set_option('display.max_rows', None) 
# Display NaN counts in each column
print(nan_counts)
pd.set_option('display.max_rows', 6) 

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [537]:
fare_sum = np.sum(df_test_pure['Fare'])
fare_av = fare_sum / df_test_pure.shape[0]
df_test['Fare'].fillna(35.54, inplace=True)

In [538]:
nan_counts = df_test_pure.isna().sum()


pd.set_option('display.max_rows', None) 
# Display NaN counts in each column
print(nan_counts)
pd.set_option('display.max_rows', 6) 

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [539]:
df_test_pure = df_test_pure.drop(columns = ['Cabin', 'Ticket', 'Name'])

In [540]:
df_test_pure = df_test_pure.dropna()

In [542]:
df_test_pure

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
...,...,...,...,...,...,...,...,...
412,1304,3,female,28.0,0,0,7.7750,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S


In [543]:
df_test_pure['Sex'] = df_test_pure['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2
df_test_pure['Embarked'] = df_test_pure['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

In [544]:
#grab passengerid here and merge with the guessing dataframe
df_test_pass = df_test_pure['PassengerId']

In [545]:
df_test_pass = pd.DataFrame(df_test_pass)

In [546]:
df_test_pure = df_test_pure.drop(columns = ['PassengerId'])
df_test_pure

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,3
1,3,2,47.0,1,0,7.0000,1
2,2,1,62.0,0,0,9.6875,3
...,...,...,...,...,...,...,...
412,3,2,28.0,0,0,7.7750,1
414,1,2,39.0,0,0,108.9000,2
415,3,1,38.5,0,0,7.2500,1


In [547]:
df_test_pure.shape

(332, 7)

In [548]:
df_test_ageNa = df_test[df_test['Age'].isna()]

df_test_ageNa

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S
22,914,1,"Flegenheim, Mrs. Alfred (Antoinette)",female,NaN,0,0,PC 17598,31.6833,NaN,S
29,921,3,"Samaan, Mr. Elias",male,NaN,2,0,2662,21.6792,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [549]:
age_pass = df_test_ageNa['PassengerId']

In [550]:
age_pass = pd.DataFrame(age_pass)
age_pass

,PassengerId
10,902
22,914
29,921
...,...
413,1305
416,1308
417,1309


In [551]:
df_test_ageNa = df_test_ageNa.drop(columns = ['Cabin', 'Ticket', 'PassengerId', 'Name'])

df_test_ageNa['Family'] = df_test['SibSp'] + df_test['Parch']

In [552]:
df_test_ageNa['Sex'] = df_test_ageNa['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2
df_test_ageNa['Embarked'] = df_test_ageNa['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})
df_test_ageNa

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
10,3,1,NaN,0,0,7.8958,1,0
22,1,2,NaN,0,0,31.6833,1,0
29,3,1,NaN,2,0,21.6792,2,2
...,...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,8.0500,1,0
416,3,1,NaN,0,0,8.0500,1,0
417,3,1,NaN,1,1,22.3583,2,2


Figuring out the age for test.csv

In [558]:
X_age = df_test_ageNa.drop(columns = ['Age'])

X_age_scaled = scaler_X.fit_transform(X_age)

In [559]:
y_pred = best_rf.predict(X_age_scaled)

y_pred = y_pred.reshape(-1, 1)


y_pred = scaler_y.inverse_transform(y_pred)


In [560]:

y_pred

array([[30.69523861],
       [31.68612469],
       [28.448407  ],
       [27.75105431],
       [26.66126614],
       [28.26730292],
       [39.0986564 ],
       [27.888585  ],
       [44.47677363],
       [29.74828075],
       [33.03639503],
       [26.83330093],
       [30.69523861],
       [41.80061309],
       [29.51872186],
       [20.2252374 ],
       [30.06357079],
       [26.83330093],
       [27.888585  ],
       [27.888585  ],
       [23.07747911],
       [20.34526583],
       [32.42791859],
       [25.32601313],
       [27.888585  ],
       [22.75276014],
       [31.10474473],
       [28.64446308],
       [36.17035148],
       [39.0986564 ],
       [30.69523861],
       [20.26286703],
       [32.42791859],
       [37.82631149],
       [32.15006608],
       [32.47518843],
       [27.888585  ],
       [ 9.73334883],
       [35.76629582],
       [27.43857886],
       [23.09975061],
       [35.79728879],
       [32.42791859],
       [21.44335566],
       [26.83330093],
       [24

In [561]:
y_pred = np.round(y_pred)

In [562]:
df_test_ageNa.shape

(86, 8)

In [563]:
y_pred.shape

(86, 1)

In [564]:
df_test_ageNa.loc[:, 'Age'] = y_pred

In [565]:
df_test_ageNa

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
10,3,1,31.0,0,0,7.8958,1,0
22,1,2,32.0,0,0,31.6833,1,0
29,3,1,28.0,2,0,21.6792,2,2
...,...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,8.0500,1,0
416,3,1,27.0,0,0,8.0500,1,0
417,3,1,30.0,1,1,22.3583,2,2


In [566]:
merged_pass = pd.concat([age_pass, df_test_pass], ignore_index=True)

In [567]:
merged_pass

,PassengerId
0,902
1,914
2,921
...,...
415,1304
416,1306
417,1307


In [568]:
merged_df_surv = pd.concat([df_test_ageNa, df_test_pure], ignore_index=True)

In [569]:
merged_df_surv['Family'] = merged_df_surv['SibSp'] + merged_df_surv['Parch']

In [570]:
merged_df_surv

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,3,1,31.0,0,0,7.8958,1,0
1,1,2,32.0,0,0,31.6833,1,0
2,3,1,28.0,2,0,21.6792,2,2
...,...,...,...,...,...,...,...,...
415,3,2,28.0,0,0,7.7750,1,0
416,1,2,39.0,0,0,108.9000,2,0
417,3,1,38.5,0,0,7.2500,1,0


In [574]:
y_pred_surv = model.predict(merged_df_surv)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [575]:
y_pred_surv

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [576]:
y_pred_surv.shape

(418,)

In [577]:
merged_df_surv.loc[:, 'Survived'] = y_pred_surv

In [581]:
merged_df_surv

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Survived
0,3,1,31.0,0,0,7.8958,1,0,0
1,1,2,32.0,0,0,31.6833,1,0,0
2,3,1,28.0,2,0,21.6792,2,2,0
...,...,...,...,...,...,...,...,...,...
415,3,2,28.0,0,0,7.7750,1,0,0
416,1,2,39.0,0,0,108.9000,2,0,0
417,3,1,38.5,0,0,7.2500,1,0,0


In [590]:
merged_pass = merged_pass.sort_values(by='PassengerId', ascending=True)
merged_pass = merged_pass.reset_index(drop=True)

In [591]:
merged_pass

,PassengerId
0,892
1,893
2,894
...,...
415,1307
416,1308
417,1309


In [592]:
final = pd.concat([merged_df_surv, merged_pass], axis=1)

final


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Survived,PassengerId
0,3,1,31.0,0,0,7.8958,1,0,0,892
1,1,2,32.0,0,0,31.6833,1,0,0,893
2,3,1,28.0,2,0,21.6792,2,2,0,894
...,...,...,...,...,...,...,...,...,...,...
415,3,2,28.0,0,0,7.7750,1,0,0,1307
416,1,2,39.0,0,0,108.9000,2,0,0,1308
417,3,1,38.5,0,0,7.2500,1,0,0,1309


In [583]:
nan_counts = final.isna().sum()


pd.set_option('display.max_rows', None) 
# Display NaN counts in each column
print(nan_counts)
pd.set_option('display.max_rows', 6) 

Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Family         0
Survived       0
PassengerId    0
dtype: int64


In [527]:
nan_counts = merged_df_surv.isna().sum()


pd.set_option('display.max_rows', None) 
# Display NaN counts in each column
print(nan_counts)
pd.set_option('display.max_rows', 6) 

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Family      0
Survived    0
dtype: int64


In [595]:
final_submit = final.drop(columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Family'])

In [596]:
final_submit

,Survived,PassengerId
0,0,892
1,0,893
2,0,894
...,...,...
415,0,1307
416,0,1308
417,0,1309


In [598]:
# Get the order of the current columns
column_order = final_submit.columns.tolist()

# Switch the positions of column1 and column2
column_order[column_order.index('PassengerId')] = 'Survived'
column_order[column_order.index('Survived')] = 'PassengerId'

# Reassign the columns in the new order
final_submit = final_submit[column_order]

final_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
...,...,...
415,1307,0
416,1308,0
417,1309,0


In [599]:
csv_file_path = '/kaggle/working/my_dataframe.csv'

# Use to_csv to export the DataFrame to a CSV file
final_submit.to_csv(csv_file_path, index=False)  # index=False to exclude the index column

print(f"CSV file saved at {csv_file_path}")

CSV file saved at /kaggle/working/my_dataframe.csv
